In [ ]:
from adam.core.computations import KinDynComputations
from adam.geometry import utils
import numpy as np
import casadi as cs
import gym_ignition_models

In [ ]:
def SX2DM(sx):
    return cs.DM(sx)

## Import the robot .urdf

In [ ]:
urdf_path = gym_ignition_models.get_model_file("iCubGazeboV2_5")
print(urdf_path)
# The joint list
joints_name_list = [
    'torso_pitch', 'torso_roll', 'torso_yaw', 'l_shoulder_pitch',
    'l_shoulder_roll', 'l_shoulder_yaw', 'l_elbow', 'r_shoulder_pitch',
    'r_shoulder_roll', 'r_shoulder_yaw', 'r_elbow', 'l_hip_pitch', 'l_hip_roll',
    'l_hip_yaw', 'l_knee', 'l_ankle_pitch', 'l_ankle_roll', 'r_hip_pitch',
    'r_hip_roll', 'r_hip_yaw', 'r_knee', 'r_ankle_pitch', 'r_ankle_roll'
]
# Specify the root link
root_link = 'root_link'
kinDyn = KinDynComputations(urdf_path, joints_name_list, root_link)

The class `KinDynComputations` contains some algorithms for computing kinematics and dynamics quantities.

In [ ]:
# Set joints and base informations
xyz = (np.random.rand(3) - 0.5) * 5
rpy = (np.random.rand(3) - 0.5) * 5
H_b = utils.H_from_PosRPY(xyz, rpy)
v_b = (np.random.rand(6) - 0.5) * 5
s = (np.random.rand(len(joints_name_list)) - 0.5) * 5
s_dot = (np.random.rand(len(joints_name_list)) - 0.5) * 5

## Mass Matrix

In [ ]:
M = kinDyn.mass_matrix_fun()
print('Mass matrix:\n', SX2DM(M(H_b, s)))

## Centroidal Momentum Matrix

In [ ]:
Jcm = kinDyn.centroidal_momentum_matrix_fun()
print('Centroidal Momentum Matrix:\n', SX2DM(Jcm(H_b, s)))

## Center of mass position

In [ ]:
CoM = kinDyn.CoM_position_fun()
print('Center of Mass position:\n', SX2DM(CoM(H_b, s)))

## Total Mass

In [ ]:
print('Total mass:\n', kinDyn.get_total_mass())

## Jacobian

In [ ]:
J = kinDyn.jacobian_fun('l_sole')
print('Jacobian of the left sole:\n', SX2DM(J(H_b, s)))

## Relative jacobian

In [ ]:
J_r = kinDyn.relative_jacobian_fun('l_sole')
print('Jacobian between the root link and left sole:\n', SX2DM(J_r(s)))

## Forward kinematics

In [ ]:
H = kinDyn.forward_kinematics_fun('l_sole')
print('Forward kinematics to the left sole:\n', SX2DM(H(H_b, s)))

## Bias force term

In [ ]:
h = kinDyn.bias_force_fun()
print('Bias force term:\n', SX2DM(h(H_b, s, v_b, s_dot)))

## Coriolis term

In [ ]:
C = kinDyn.coriolis_term_fun()
print('Coriolis force term:\n', SX2DM(C(H_b, s, v_b, s_dot)))

## Gravity term

In [ ]:
G = kinDyn.gravity_term_fun()
print('Gravity term:\n', SX2DM(G(H_b, s)))